# HPH第一版实验

### 思路

*id统计相关*
1. item_id, shop_id的交易量, 交易率
2. item_id, shop_id7天是否新出现, 3天是否新出现, 昨天是否新出现


In [123]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
offline_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt', sep=' ')
# online_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt', sep=' ')

# 1 数据集统一处理

In [124]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

offline_df['date'] = offline_df['context_timestamp'].apply(lambda x: extract_date(x))
offline_df['weekday'] = offline_df['context_timestamp'].apply(lambda x: extract_weekday(x))
offline_df['hour'] = offline_df['context_timestamp'].apply(lambda x: extract_hour(x))

In [125]:
# id是否重复信息
# le = preprocessing.LabelEncoder()
# offline_df['item_id_dup'] = le.fit_transform(offline_df['item_id'].duplicated())
# offline_df['shop_id_dup'] = le.fit_transform(offline_df['shop_id'].duplicated())
# offline_df['item_brand_id_dup'] = le.fit_transform(offline_df['item_brand_id'].duplicated())
# offline_df['user_id_dup'] = le.fit_transform(offline_df['user_id'].duplicated())



In [143]:
%run ../util/time_utils.py

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 3)

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 1)


In [144]:
# 统计信息附加

def getColTradeRate(df, idCol):
    rateCol = idCol + '_tr'
    pvCol = idCol + '_pv'
    try:
        del df[rateCol]
        del df[pvCol]
    except:
        pass
    a = offline_df.groupby([idCol]).agg({'is_trade':'sum'})
    b = offline_df.groupby([idCol]).agg({'is_trade':'size'})
    c = a.join(b, lsuffix="_sum", rsuffix="_size")
    c[rateCol] = c['is_trade_sum'] / c['is_trade_size']
    c[pvCol] = c['is_trade_size']
    return offline_df.join(c[[rateCol, pvCol]], on=idCol)
    

# 各类id的 交易量, 交易率
offline_df = getColTradeRate(offline_df, 'item_city_id')
offline_df = getColTradeRate(offline_df, 'item_id')
offline_df = getColTradeRate(offline_df, 'shop_id')
offline_df = getColTradeRate(offline_df, 'item_brand_id')
offline_df = getColTradeRate(offline_df, 'user_occupation_id')


In [145]:
# 分桶
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

offline_df['item_city_id_pv_cut'] = pd.cut(offline_df['item_city_id_pv'].values, bins=10, retbins=False)
offline_df['item_id_pv_cut'] = pd.cut(offline_df['item_id_pv'].values, bins=10, retbins=False)
offline_df['shop_id_pv_cut'] = pd.cut(offline_df['shop_id_pv'].values, bins=10, retbins=False)
offline_df['item_brand_id_pv_cut'] = pd.cut(offline_df['item_brand_id_pv'].values, bins=10, retbins=False)
offline_df['user_occupation_id_pv_cut'] = pd.cut(offline_df['user_occupation_id_pv'].values, bins=10, retbins=False)

offline_df['item_city_id_pv_cut'] = le.fit_transform(offline_df['item_city_id_pv_cut'].copy())
offline_df['item_id_pv_cut'] = le.fit_transform(offline_df['item_id_pv_cut'].copy())
offline_df['shop_id_pv_cut'] = le.fit_transform(offline_df['shop_id_pv_cut'].copy())
offline_df['item_brand_id_pv_cut'] = le.fit_transform(offline_df['item_brand_id_pv_cut'].copy())
offline_df['user_occupation_id_pv_cut'] = le.fit_transform(offline_df['user_occupation_id_pv_cut'].copy())

In [146]:
# standardization and scaling

from sklearn import preprocessing

def scale(df, idCol):
    min_max_scaler = preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df[idCol].values.reshape(-1, 1))
    return pd.Series(a.reshape(1, -1)[0])

# 令数据再少一些?
offline_df['shop_score_service_scaled'] = scale(offline_df, 'shop_score_service')
offline_df['shop_score_delivery_scaled'] = scale(offline_df, 'shop_score_delivery')
offline_df['shop_score_description_scaled'] = scale(offline_df, 'shop_score_description')
offline_df['shop_review_positive_rate_scaled'] = scale(offline_df, 'shop_review_positive_rate')
offline_df['item_city_id_tr_scaled'] = scale(offline_df, 'item_city_id_tr')
offline_df['item_id_tr_scaled'] = scale(offline_df, 'item_id_tr')
offline_df['shop_id_tr_scaled'] = scale(offline_df, 'shop_id_tr')
offline_df['item_brand_id_tr_scaled'] = scale(offline_df, 'item_brand_id_tr')
offline_df['user_occupation_id_scaled'] = scale(offline_df, 'user_occupation_id_tr')

# offline_df = offline_df.dropna()

# 2 生成矩阵数据

In [157]:
used = 78261
X = offline_df[
    [
       'date', 
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
]
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']]

# 3 kfold 测试

In [158]:
train_input = X
train_input['is_trade'] = y

%run ../yuhua/BaseFrame.py

def tatx(train_df, test_df, valid_df):

    X_train = train_df.copy()
    del X_train['is_trade']

    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    X_valid = valid_df.copy()
    del X_valid['is_trade']
    y_valid = valid_df['is_trade']
    
    import xgboost
    params = {
        'nthread': 20
    }
    m = xgboost.XGBClassifier(**params)
    m.fit(X_train, y_train)

    from sklearn.metrics import log_loss
    l_train = log_loss(y_train, m.predict_proba(X_train))
    l_test = log_loss(y_test, m.predict_proba(X_test))
    l_valid = log_loss(y_valid, m.predict_proba(X_valid))

    return l_train, l_test, l_valid

frame = BaseFrame(train_input, 0.05)
frame.kfold_by_date('date', tatx)
# 0.077189, 0.076092
# test = 0.077047  valid = 0.07597
# test = 0.077077  valid = 0.075944

/Users/prehawkmac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2018-09-18 to 2018-09-18 is training set, 2018-09-19 is test set, 2018-09-24 is valid set, start training ... 
0 done, logloss train = 0.08453301421 (compare to previous: 0.08453301421), 
                    	test = 0.0860219639715 (compare to previous: 0.0860219639715), 
                    	valid = 0.0764235695799 (compare to previous: 0.0764235695799)

2018-09-18 to 2018-09-19 is training set, 2018-09-20 is test set, 2018-09-24 is valid set, start training ... 
1 done, logloss train = 0.0850018344811 (compare to previous: 0.000468820271079), 
                    	test = 0.0850253621848 (compare to previous: -0.00099660178662), 
                    	valid = 0.0762046606052 (compare to previous: -0.000218908974612)

2018-09-18 to 2018-09-20 is training set, 2018-09-21 is test set, 2018-09-24 is valid set, start training ... 
2 done, logloss train = 0.0849235743849 (compare to previous: -7.82600961871e-05), 
                    	test = 0.0842971726149 (compare to previous: -0.000728189